In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qqq

     |████████████████████████████████| 1.8MB 11.4MB/s 
     |████████████████████████████████| 102kB 13.8MB/s 
     |████████████████████████████████| 163kB 52.4MB/s 
     |████████████████████████████████| 133kB 56.5MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 


In [3]:
from wandb.keras import WandbCallback
import wandb

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import tensorflow as tf

# Load dataset as dataframe
base_path = '/content/drive/My Drive/Y4 Sem 1/NN Project/'
df = pd.read_csv('/content/drive/My Drive/Y4 Sem 1/NN Project/aligned_gender.txt', sep='\t')
df['datadir'] = base_path + df['datadir'].astype(str)

# Train test split
train_val_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_val_df, test_size=0.2)

# Fix seed
seed = 7
np.random.seed(seed)
tf.random.set_seed(seed)

# default hyperparameters
defaults = {
    'epochs': 20,
    'batch_size': 64,
    'learning_rate': 5e-4,
    'weight_decay': 1e-9,
    'fc1_num_neurons': 512,
    'fc2_num_neurons': 512,
    'fc3_num_neurons': 512,
    'dropout1_rate': 0,
    'dropout2_rate': 0,
    'dropout3_rate': 0,
    'seed': seed,
    'optimizer': 'adam',
    'hidden_activation': 'relu',
    'output_activation': 'sigmoid',
    'loss_function': 'binary_crossentropy',
    'metrics': ['accuracy'],
}

wandb.init(
    config=defaults,
    name='Decay 1e-9',
    project='nn_project_tuning',
)
config = wandb.config

# Define model
mobile_net_v2 = tf.keras.applications.MobileNetV2(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    input_shape=(224,224,3),
)

fc1 = tf.keras.layers.Dense(
    config.fc1_num_neurons,
    activation=config.hidden_activation,
    kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
)

fc2 = tf.keras.layers.Dense(
    config.fc2_num_neurons,
    activation=config.hidden_activation,
    kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
)

fc3 = tf.keras.layers.Dense(
    config.fc3_num_neurons,
    activation=config.hidden_activation,
    kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
)

bn1 = tf.keras.layers.BatchNormalization()
bn2 = tf.keras.layers.BatchNormalization()
bn3 = tf.keras.layers.BatchNormalization()
bn4 = tf.keras.layers.BatchNormalization()

dropout1 = tf.keras.layers.Dropout(rate=config.dropout1_rate)
dropout2 = tf.keras.layers.Dropout(rate=config.dropout2_rate)
dropout3 = tf.keras.layers.Dropout(rate=config.dropout3_rate)

model = tf.keras.models.Sequential([
    mobile_net_v2,
    tf.keras.layers.Flatten(),
    bn1,
    fc1,
    dropout1,
    bn2,
    fc2,
    dropout2,
    bn3,
    fc3,
    dropout3,
    bn4,
    tf.keras.layers.Dense(1, activation=config.output_activation),
])

# Compile model 
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    loss=config.loss_function,
    metrics=config.metrics,
)

# Load images into keras image generator 
datagen_train = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)
datagen_val = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)
datagen_test = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)

train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

val_generator = datagen_train.flow_from_dataframe(
    dataframe=val_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

test_generator = datagen_test.flow_from_dataframe(
    dataframe=test_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=config.epochs,
    callbacks=[WandbCallback()],
)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


9412608/9406464 [==============================] - 0s 0us/step
Found 7804 validated image filenames.
Found 1951 validated image filenames.
Found 2439 validated image filenames.
Epoch 1/20
122/122 [==============================] - 2008s 16s/step - loss: 0.3880 - accuracy: 0.8257 - val_loss: 0.4187 - val_accuracy: 0.8319
Epoch 2/20
122/122 [==============================] - 135s 1s/step - loss: 0.1712 - accuracy: 0.9312 - val_loss: 0.2868 - val_accuracy: 0.9052
Epoch 3/20
122/122 [==============================] - 133s 1s/step - loss: 0.0882 - accuracy: 0.9664 - val_loss: 0.7469 - val_accuracy: 0.8206
Epoch 4/20
122/122 [==============================] - 128s 1s/step - loss: 0.0774 - accuracy: 0.9725 - val_loss: 0.3497 - val_accuracy: 0.8975
Epoch 5/20
122/122 [==============================] - 127s 1s/step - loss: 0.0595 - accuracy: 0.9774 - val_loss: 0.7597 - val_accuracy: 0.8452
Epoch 6/20
122/122 [==============================] - 125s 1s/step - loss: 0.0455 - accuracy: 0.9831 - val